In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
import functools
import collections
import datetime as dt
import sklearn
from sklearn import metrics
import statsmodels
from statsmodels import stats
from statsmodels.stats import inter_rater

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
path_records = "R:\\VPHI\\Welfare\\2- Research Projects\\OFHE2.OriginsE2\\Ganter system validation\\RECORDS"
df = pd.read_csv(os.path.join(path_records,'VideoObservation','VideoObservationAllStudents.csv'), sep=';') 
li_student = df['worker'].unique()
df['VideoAnalyse'] = df['VideoAnalyse'].astype('category')
print('There is %d students involved'%len(li_student))
print(df.shape)
df.head(3)

There is 5 students involved
(1025, 14)


,Date,batch,Heure,TagID,BirdID,trackZone,signalstrength,VideoAnalyse,Comment,worker,Pen,backpackID,Obszone_whenWrong,obsZone
0,07.08.2020,1,02:09:58,02001E2F,130.0,Tier 1,17.0,1,NaN,jonas,9.0,sws,NaN,Tier 1
1,07.08.2020,1,02:29:29,02001E2F,130.0,Tier 2 + untere Stange,15.0,1,18 seconds before BG,jonas,9.0,sws,NaN,Tier 2 + untere Stange
2,07.08.2020,1,02:36:16,02001E2F,130.0,Tier 2 (mini 9),10.0,1,NaN,jonas,9.0,sws,NaN,Tier 2 (mini 9)


In [3]:
df['VideoAnalyse'].iloc[0]

1

In [4]:
#TODO: handle 2, -1 situations!!
df = df[df['VideoAnalyse'].isin([0,1])]
print(df.shape)
df.head(3)

(973, 14)


,Date,batch,Heure,TagID,BirdID,trackZone,signalstrength,VideoAnalyse,Comment,worker,Pen,backpackID,Obszone_whenWrong,obsZone
0,07.08.2020,1,02:09:58,02001E2F,130.0,Tier 1,17.0,1,NaN,jonas,9.0,sws,NaN,Tier 1
1,07.08.2020,1,02:29:29,02001E2F,130.0,Tier 2 + untere Stange,15.0,1,18 seconds before BG,jonas,9.0,sws,NaN,Tier 2 + untere Stange
2,07.08.2020,1,02:36:16,02001E2F,130.0,Tier 2 (mini 9),10.0,1,NaN,jonas,9.0,sws,NaN,Tier 2 (mini 9)


In [5]:
#handle duplicates row
df_duplicates = df.groupby(['Date','Heure','BirdID','trackZone','Pen','worker']).agg(lambda x: list(x)).reset_index()
df_duplicates['nbr'] = df_duplicates['trackZone'].map(lambda x: len(x))
df_duplicates[df_duplicates['nbr']>1]
#--> keep first entry!
df = df.drop_duplicates(subset=['Date','Heure','BirdID','trackZone','Pen','worker'], keep='first', inplace=False)

In [6]:
df_raters = df.pivot(index=['Date','Heure','BirdID','trackZone','Pen'], columns='worker', 
                     values='VideoAnalyse').reset_index()
print(df_raters.shape)
df_raters.head(3)

(672, 10)


worker,Date,Heure,BirdID,trackZone,Pen,Ana,charlene,doriana,jonas,nabila
0,04.08.2020,02:10:45,8.0,Tier 3 Rampe + Nestbox,12.0,NaN,0,NaN,NaN,NaN
1,04.08.2020,02:11:05,8.0,Tier 1,12.0,NaN,1,NaN,NaN,NaN
2,04.08.2020,02:13:13,20.0,Tier 3 Rampe + Nestbox,5.0,NaN,1,NaN,NaN,1


In [7]:
df_raters['li_workers'] = df_raters.apply(lambda x: [i for i in li_student if math.isnan(x[i])==False], axis=1)
df_raters['nbr_workers'] = df_raters['li_workers'].map(lambda x: len(x))
df_raters['li_workers'] = df_raters['li_workers'].map(lambda x: '_'.join(sorted(x)))
df_raters[df_raters['nbr_workers']>1].head(5)

worker,Date,Heure,BirdID,trackZone,Pen,Ana,charlene,doriana,jonas,nabila,li_workers,nbr_workers
2,04.08.2020,02:13:13,20.0,Tier 3 Rampe + Nestbox,5.0,NaN,1,NaN,NaN,1,charlene_nabila,2
4,04.08.2020,02:15:22,20.0,Tier 2 + untere Stange,5.0,NaN,1,NaN,NaN,1,charlene_nabila,2
5,04.08.2020,02:17:15,14.0,Tier 2 (mini 3),3.0,NaN,1,1,1,1,charlene_doriana_jonas_nabila,4
6,04.08.2020,02:17:34,14.0,Tier 2 + untere Stange,3.0,1,1,1,1,1,Ana_charlene_doriana_jonas_nabila,5
14,04.08.2020,02:19:08,68.0,Tier 3 Rampe + Nestbox,4.0,NaN,1,1,NaN,NaN,charlene_doriana,2


In [8]:
#nbr of observation in commun and not in commun
df_raters.groupby(['li_workers'])['Heure'].agg(lambda x: len(x)).reset_index()

,li_workers,Heure
0,Ana,252
1,Ana_charlene,43
2,Ana_charlene_doriana_jonas_nabila,24
3,charlene,38
4,charlene_doriana,78
5,charlene_doriana_jonas_nabila,1
6,charlene_jonas,30
7,charlene_nabila,35
8,doriana,18
9,jonas,122


In [9]:
#save 3 workers for reliability
for i, df_ in df_raters[df_raters['nbr_workers']==2].groupby(['li_workers']):
    print(i)
    df_.to_csv(os.path.join(path_records,'VideoObservation','df_reliability_2workers_'+i+'.csv'), 
                                              index=False, sep=';')
#save 5 raters for reliability  
df_raters[df_raters['nbr_workers']==5].to_csv(os.path.join(path_records,'VideoObservation','df_reliability_5workers.csv'), 
                                              index=False, sep=';')

Ana_charlene
charlene_doriana
charlene_jonas
charlene_nabila


In [10]:
#print the rest to correct manually
df_raters[df_raters['nbr_workers'].isin([3,4])]

worker,Date,Heure,BirdID,trackZone,Pen,Ana,charlene,doriana,jonas,nabila,li_workers,nbr_workers
5,04.08.2020,02:17:15,14.0,Tier 2 (mini 3),3.0,NaN,1,1,1,1,charlene_doriana_jonas_nabila,4
